In [1]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred
import config

# ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)
# ge_genus_stage = load.loadGEOverlappingTCMA("genus", includeStage=True)
# aak_ge_stage = load.loadGEWithClinical(includeStage=True)

In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred
import config

# ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)
# ge_genus_stage = load.loadGEOverlappingTCMA("genus", includeStage=True)
# aak_ge_stage = load.loadGEWithClinical(includeStage=True)

In [ ]:
for target in config.prediction_targets:
    data, files = load.loadAll(includeStage=(target=="stage"), sameSamples=True)
    
    for sampling in config.sampling:
        pred.runExperiments(data[:], files[:], target=target, sampling=sampling)

In [2]:
data, files = load.loadAll(includeStage=True, sameSamples=True)

c:\Users\Ameno\Desktop\Thesis\Host-Microbial-Omics-Integration\loader.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aak_ge_clinical_types["portion_id"] = aak_ge_clinical_types.apply(lambda row: str(row.portion_id)[:-4], axis=1)
c:\Users\Ameno\Desktop\Thesis\Host-Microbial-Omics-Integration\loader.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aak_ge_clinical_types.drop_duplicates(["portion_id"], inplace=True)
c:\Users\Ameno\Desktop\Thesis\Host-Microbial-Omics-Integration\loader.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [ ]:
pred.runExperiments(data[:], files[:], target="stage", sampling="random_sampling")

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score, log_loss
import numpy as np

d = data[1]
d = load.attachStageStatus(d)
x, y = pr.splitData(d, target="stage", project="ESCA")
print(y)
model = LinearRegression()
# pred.runRandomSampling(x, y, model)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, stratify=y, random_state=42)

model.fit(x_train, y_train)
y_predicted = model.predict(x_test)
y_predicted_rounded = np.rint(y_predicted)
y_predicted_rounded
y_predicted


TCGA-IG-A3I8-01    2
TCGA-IG-A3I8-11    0
TCGA-IG-A3QL-01    2
TCGA-IG-A3Y9-01    3
TCGA-IG-A3YA-01    3
                  ..
TCGA-LN-A4A8-01    2
TCGA-LN-A4MR-01    2
TCGA-LN-A5U5-01    4
TCGA-LN-A8I1-01    2
TCGA-LN-A9FQ-01    2
Name: stage, Length: 62, dtype: int32


array([ 1.93313068,  2.18738136,  2.03187015,  2.24287305,  0.00728352,
        2.30506897,  2.35438891,  2.05198344,  2.11761247,  1.92952112,
        2.29942078,  2.02054195, -0.13128815,  2.3923181 ,  2.21638903,
        1.6062588 ,  1.97067496,  2.06405305,  1.99515629,  1.82392603,
       -0.05705918,  2.23732039,  2.06897079,  1.88521266,  2.22694542])

In [7]:
model.coef_
# features_with_coefficients = pd.DataFrame(zip(x_train.columns, model.coef_))
features_with_coefficients = pd.DataFrame({"feature":x_train.columns,"coefficients":np.transpose(model.coef_)})
features_with_coefficients_abs = features_with_coefficients.copy()
features_with_coefficients_abs["coefficients"] = features_with_coefficients_abs.apply(lambda row: abs(row.coefficients), axis=1)
features_with_coefficients_abs
sorted_features = features_with_coefficients_abs.sort_values("coefficients", ascending=False)
top_features = sorted_features.head(20)["feature"].values
x_train[top_features]

best_indices = [x_train.columns.get_loc(c) for c in x_train.columns if c in top_features]


print(x_train[top_features].columns)

print(x_train.iloc[:, best_indices].columns)
print(x_train.iloc[:, pr.selectFeatures(x, y, 20, "linreg")].columns)

Index(['XIST', 'RPS4Y1', 'ZFY', 'DDX3Y', 'CAPN9', 'KDM5D', 'PRKY', 'UGT1A7',
       'MAGEA9B', 'NLGN4Y', 'USP9Y', 'EIF1AY', 'CDKN2A', 'FOXD1', 'IRX1',
       'HOXA6', 'MAGEA1', 'UGT2B15', 'HOXA1', 'YBX2'],
      dtype='object')
Index(['RPS4Y1', 'XIST', 'DDX3Y', 'KDM5D', 'USP9Y', 'EIF1AY', 'ZFY', 'NLGN4Y',
       'PRKY', 'UGT2B15', 'IRX1', 'CAPN9', 'FOXD1', 'CDKN2A', 'YBX2',
       'MAGEA9B', 'MAGEA1', 'UGT1A7', 'HOXA6', 'HOXA1'],
      dtype='object')
Index(['RPS4Y1', 'XIST', 'KDM5D', 'EIF1AY', 'GSTM1', 'ZFY', 'CYorf15A',
       'NLRP2', 'NLGN4Y', 'PPAN-P2RY11', 'PRKY', 'PRSS21', 'MUC2', 'IGF2BP3',
       'MAGEA4', 'CGREF1', 'MAGEA9B', 'FADS6', 'MAGEA1', 'ZNF883'],
      dtype='object')
